In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/train.csv")

df_test=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/test.csv")

df_sub=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")

In [ ]:
display(df_train.head())
display(df_test.head())
display(df_sub.head())

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(df_train.corr(), annot=True)

In [ ]:
feature_corr = df_train.corr().unstack().sort_values()
feature_corr

In [ ]:
feature_corr[(feature_corr>0.7)&(feature_corr<1)]

In [ ]:
df_train = df_train.drop(columns=['id'])

In [ ]:
df_train.columns[df_train.dtypes!='object']

In [ ]:
features = df_train.columns[df_train.dtypes!='object']

def plot(feature):
    plt.hist(df_train[feature])
    plt.title(feature)
    plt.show()
        
for i in features:
    plot(i)

In [ ]:
df_train.skew()

In [ ]:
manipulation_data = df_train.copy()

In [ ]:
feauture_transform = manipulation_data.loc[:,['cont0','cont4','cont5','cont6','cont7','cont8']]

In [ ]:
def log(feature):
    manipulation_data[feature] = np.log(manipulation_data[feature]+1)
        
for i in feauture_transform.columns:
    log(i)

In [ ]:
featuress = manipulation_data.columns[manipulation_data.dtypes!='object']

def plott(feature):
    plt.hist(manipulation_data[feature])
    plt.title(feature)
    plt.show()
        
for i in featuress :
    plott(i)


In [ ]:
manipulation_data['cont4'] = np.sqrt(manipulation_data ['cont4'])
manipulation_data['cont6'] = np.sqrt(manipulation_data ['cont6'])
manipulation_data['cont7'] = np.sqrt(manipulation_data['cont7'])

In [ ]:
manipulation_data .skew()

In [ ]:
train_data = pd.get_dummies(manipulation_data , drop_first=True)

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
x = train_data.drop('target', axis=1)
y = pd.DataFrame(train_data['target'])

In [ ]:
x.head()


In [ ]:
y.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

In [ ]:
#Normalization: (X-X.min())/(X.max()-X.min())
#Standardization: (X-X.mean())/X.std()

scaler_x = MinMaxScaler().fit(x_train)
scaler_y = MinMaxScaler().fit(y_train)


In [ ]:
x_train_sc = scaler_x.transform(x_train)
x_test_sc = scaler_x.transform(x_test)

In [ ]:
y_train_sc = scaler_y.transform(y_train)
y_test_sc = scaler_y.transform(y_test)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train_sc, y_train_sc)
y_pred_sc = lr_model.predict(x_test_sc)

In [ ]:
mae = mean_absolute_error(y_test_sc, y_pred_sc)
rmse = np.sqrt(mean_squared_error(y_test_sc, y_pred_sc))

print('MAE = ', mae.round(4))
print('RMSE = ', rmse.round(4))

In [ ]:
y_test_inv = scaler_y.inverse_transform(y_test_sc.reshape(-1,1))
y_pred_inv = scaler_y.inverse_transform(y_pred_sc.reshape(-1,1))

actual_mae = mean_absolute_error(y_test_inv, y_pred_inv)
actual_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

print('Actual MAE = ', int(actual_mae))
print('Actual RMSE = ', int(actual_rmse))

In [ ]:
df_test=  df_test.drop(columns=['id'])


In [ ]:
test_features = df_test.loc[:,['cont0','cont4','cont5','cont6','cont7','cont8']]

In [ ]:
def log(feature):
    df_test[feature] = np.log(df_test[feature]+1)
        
for i in test_features.columns:
    log(i)
    
df_test.skew()

In [ ]:
df_test['cont4'] = np.sqrt(df_test['cont4'])
df_test['cont6'] = np.sqrt(df_test['cont6'])
df_test['cont7'] = np.sqrt(df_test['cont7'])
df_test.skew()

In [ ]:
df_test = pd.get_dummies(df_test, drop_first=True)

In [ ]:
df_test.head()

In [ ]:
df_test.insert(loc=30,
          column='cat6_G',
          value=0)

In [ ]:
Scaled_data = scaler_x.transform(df_test)

In [ ]:
Scaled_data_pred = lr_model.predict(Scaled_data)

In [ ]:
Scaled_data_pred_inv = scaler_y.inverse_transform(Scaled_data_pred.reshape(-1,1))

In [ ]:
df_test=pd.read_csv(r"/kaggle/input/tabular-playground-series-feb-2021/test.csv")
output = pd.DataFrame()
output['id']=df_test.id
output['target']=Scaled_data_pred_inv 
output.index=df_test.id
output.to_csv('MY_Submission.csv')